This idea of this notebook is to create a first (simple) DL model using all the same features used in the multinomial logit model.

Much inspiration was derived from https://towardsdatascience.com/use-machine-learning-to-predict-horse-racing-4f1111fb6ced.

In [9]:
import math
from importlib import reload
import deeplearninglib
reload(deeplearninglib)
from deeplearninglib import *

import wandb

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

Using cpu device


In [10]:
# select model to train

model_inventory = {'mktprob': {'X_columns': ["mkt_prob"],
                               'learning_rate': 10e-1,
                               'epochs': 50,
                               'vacant_stall_indicator': False,
                               'bias': True,
                               'model_architecture': LinSig},
                   'AlunOwen_v0': {'X_columns': ["age", "sire_sr", "dam_sr", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "visor", "cheekpieces", "tonguetie"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': True,
                                   'model_architecture': LinSig},
                   'AlunOwen_v1': {'X_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "visor", "cheekpieces", "tonguetie"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': True,
                                   'model_architecture': LinSig},
                   'AlunOwen_v2': {'X_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "visor", "cheekpieces", "tonguetie"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': True,
                                   'model_architecture': LinDropReluLinSoft},
                   'AlunOwen_v3': {'X_columns': ["age", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "visor", "cheekpieces", "tonguetie"],
                                   'learning_rate': 10e-3,
                                   'epochs': 100,
                                   'vacant_stall_indicator': False,
                                   'bias': True,
                                   'model_architecture': CustLinSoft},
}

model = 'AlunOwen_v2'
X_columns = model_inventory[model]['X_columns']
learning_rate = model_inventory[model]['learning_rate']
epochs = model_inventory[model]['epochs']
vacant_stall_indicator = model_inventory[model]['vacant_stall_indicator']
bias = model_inventory[model]['bias']
model_architecture = model_inventory[model]['model_architecture']

In [11]:
# read in data

y_columns = ["win"]

train_data_fn = "data\\runners_train.csv"
test_data_fn = "data\\runners_test.csv"

train_data = RacesDataset(train_data_fn, X_columns, y_columns, vacant_stall_indicator=vacant_stall_indicator)
test_data = RacesDataset(test_data_fn, X_columns, y_columns, vacant_stall_indicator=vacant_stall_indicator, scalar=train_data.scalar)

train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [12]:
pd.options.display.max_columns = 1000 # was 20
train_data.races.iloc[:, train_data.X_columns].head()

age                                                              \
stall_number   1    2    3    4    5    6    7    8    9    10   11   12   13   
race_id                                                                         
11504         3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  0.0  0.0  0.0  0.0  0.0   
11505         4.0  6.0  9.0  7.0  5.0  8.0  5.0  5.0  5.0  4.0  5.0  0.0  0.0   
11506         3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  3.0  0.0  0.0  0.0   
11507         5.0  4.0  4.0  4.0  4.0  4.0  8.0  4.0  4.0  0.0  0.0  0.0  0.0   
11508         7.0  4.0  5.0  4.0  5.0  4.0  5.0  5.0  4.0  0.0  0.0  0.0  0.0   

                            blinkers                                          \
stall_number   14   15   16       1    2    3    4    5    6    7    8    9    
race_id                                                                        
11504         0.0  0.0  0.0      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11505         0.0  0.0  0.0      0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0   
11506         0.0  0.0  0.0      1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0   
11507         0.0  0.0  0.0      0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0   
11508         0.0  0.0  0.0      1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                                cheekpieces                 \
stall_number   10   11   12   13   14   15   16          1    2    3    4    
race_id                                                                      
11504         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  0.0   
11505         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  0.0   
11506         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  0.0   
11507         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  1.0  0.0   
11508         0.0  0.0  0.0  0.0  0.0  0.0  0.0         0.0  1.0  1.0  0.0   

                                                                          \
stall_number   5    6    7    8    9    10   11   12   13   14   15   16   
race_id                                                                    
11504         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11505         0.0  0.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   
11506         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11507         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11508         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

             daysLTO                                                          \
stall_number      1      2      3      4      5      6      7      8      9    
race_id                                                                        
11504          365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0    0.0   
11505          365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0   
11506          365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0   
11507          365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0   
11508          365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0  365.0   

                                                    entire                 \
stall_number     10     11   12   13   14   15   16     1    2    3    4    
race_id                                                                     
11504           0.0    0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0  0.0   
11505         365.0  365.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0  0.0   
11506         365.0    0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0  0.0   
11507           0.0    0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0  0.0   
11508           0.0    0.0  0.0  0.0  0.0  0.0  0.0    0.0  0.0  0.0  0.0   

                                                                          \
stall_number   5    6    7    8    9    10   11   12   13   14   15   16   
race_id                                                                    
1150

In [13]:
# build the neural network

output_layer_nodes = train_data.y.shape[1]
input_layer_nodes = train_data.X.shape[1]

# torch.manual_seed(0)
net = model_architecture(input_layer_nodes, output_layer_nodes, bias=bias).to(device) # linear-relu-linear-softwax nn (1 hidden layer)
print(f"Model structure: {model}")

for name, param in net.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param} \n")

Model structure: AlunOwen_v2
Layer: neural_network.0.weight | Size: torch.Size([73, 336]) | Values : Parameter containing:
tensor([[-0.0534,  0.0083,  0.0087,  ...,  0.0346, -0.0076,  0.0140],
        [-0.0249, -0.0526,  0.0198,  ..., -0.0133,  0.0515, -0.0225],
        [-0.0327, -0.0248, -0.0433,  ..., -0.0057,  0.0201,  0.0043],
        ...,
        [-0.0252,  0.0244, -0.0389,  ...,  0.0137,  0.0325, -0.0340],
        [-0.0041,  0.0132, -0.0151,  ...,  0.0111,  0.0313,  0.0204],
        [-0.0182,  0.0003,  0.0015,  ..., -0.0249, -0.0008, -0.0042]],
       requires_grad=True) 

Layer: neural_network.0.bias | Size: torch.Size([73]) | Values : Parameter containing:
tensor([-0.0095,  0.0282,  0.0257,  0.0228, -0.0248,  0.0464,  0.0420, -0.0320,
         0.0377, -0.0206,  0.0038,  0.0383,  0.0419,  0.0424,  0.0336,  0.0083,
         0.0179,  0.0124,  0.0101, -0.0337, -0.0399, -0.0384, -0.0011,  0.0140,
         0.0304, -0.0003,  0.0221, -0.0120, -0.0010, -0.0466,  0.0355,  0.0161,
       

In [14]:
# example to show how model is used from prediction

X = torch.rand(1, input_layer_nodes, device=device)
logits = net(X)
y_pred = logits.argmax(1)
print(f"Predicted class: {y_pred}")
print(X.shape)

Predicted class: tensor([2])
torch.Size([1, 336])


In [15]:
%env WANDB_NOTEBOOK_NAME 'C:\Users\gille\OneDrive\1-Projects\_Horse Racing 2H22\New Framework\3b_Deep Learning.ipynb'

# optimizing model parameters

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="horse-racing-project",
    
    # track hyperparameters and run metadata
    config={
    "device": device,
    "model": model,
    "X_columns": X_columns,
    "learning_rate": learning_rate,
    "epochs": epochs,
    "vacant_stall_indicator": vacant_stall_indicator,
    "bias": bias,
    "model_architecture": list(net.modules())
    }
)

# initialize the loss function
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, net, loss_fn, optimizer, device)
    (acc, loss) = test_loop(test_dataloader, net, loss_fn, device)
    wandb.log({"acc": acc, "loss": loss})
print("Done!")

# finish the wandb run, necessary in notebooks
wandb.finish()

env: WANDB_NOTEBOOK_NAME='C:\Users\gille\OneDrive\1-Projects\_Horse Racing 2H22\New Framework\3b_Deep Learning.ipynb'


Epoch 1
-------------------------------
loss: 2.770813  [   64/16620]
loss: 2.765894  [ 6464/16620]
loss: 2.806000  [12864/16620]
Test Error: 
 Accuracy: 10.5%, Avg loss: 2.773996 

Epoch 2
-------------------------------
loss: 2.770087  [   64/16620]
loss: 2.764469  [ 6464/16620]
loss: 2.803839  [12864/16620]
Test Error: 
 Accuracy: 11.4%, Avg loss: 2.772232 

Epoch 3
-------------------------------
loss: 2.769843  [   64/16620]
loss: 2.763364  [ 6464/16620]
loss: 2.802976  [12864/16620]
Test Error: 
 Accuracy: 11.7%, Avg loss: 2.770560 

Epoch 4
-------------------------------
loss: 2.766474  [   64/16620]
loss: 2.761896  [ 6464/16620]
loss: 2.801050  [12864/16620]
Test Error: 
 Accuracy: 11.8%, Avg loss: 2.768826 

Epoch 5
-------------------------------
loss: 2.766091  [   64/16620]
loss: 2.760155  [ 6464/16620]
loss: 2.798855  [12864/16620]
Test Error: 
 Accuracy: 12.2%, Avg loss: 2.767065 

Epoch 6
-------------------------------
loss: 2.763924  [   64/16620]
loss: 2.757107  [ 64

acc,▁▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇██████▇█
loss,██▇▇▆▆▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
acc,0.18775
loss,2.68749


In [16]:
for para_name, para_vals in net.named_parameters():
    np.savetxt(para_name + ".csv", para_vals.data.numpy(), fmt='%6.3f', delimiter=",")